### Index Service

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
print(os.getenv('v_api_key'))


Creating index on vectara corpus

In [ ]:

import requests
import json

url = "https://api.vectara.io/v1/index"

payload = json.dumps({
  "customerId": os.getenv('customer_id'),
  "corpusId": 3,
  "document": {
    "documentId": "energy.pdf",
    "title": "Untitled document",
    "description": "This dicument is related to energy meter",
    "metadataJson": "{\"author\": \"Inam\"}",
    "section": [
      {
        "id": 1,
        "title": "string",
        "text": "The text for this section",
        "metadataJson": "{\"author\": \"Inam\"}",
      }
    ]
  }
})
headers = {
  'Content-Type': 'application/json',
  'customer-id': os.getenv('customer_id'),
  'Accept': 'application/json',
  'x-api-key': os.getenv('v_api_key')
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

Creating index using file upload api on vectara

In [48]:
# import requests

# # url = "https://api.vectara.io/v1/upload"
# url =  "https://api.vectara.io/v1/upload"

# payload = {
#   "customerId": os.getenv('customer_id'),
#   "corpusId": 
# }
# # payload = {}
# files=[
#   ('file',('file',open('C:/Users/Dell/Downloads/Documents/energy meter.pdf','rb'),'application/octet-stream'))
# ]

# headers = {
#   'Content-Type': 'multipart/form-data',
#   'Accept': 'application/json',
#   'x-api-key': os.getenv('v_api_key')
# }

# response = requests.request("POST", url, headers=headers, files=files, data=payload)

# print(response.text)

{"httpCode":400,"internalCode":3,"details":"Request entity too large.","status":{}}


In [58]:
import requests
files=[
  ('file',('file',open('C:/Users/Dell/Downloads/Documents/energy meter.pdf','rb'),'application/octet-stream'))
]

post_headers = {
"x-api-key": os.getenv('v_api_key')
}
url = f"https://api.vectara.io/v1/upload?c={os.getenv('customer_id')}&o={3}"
response = requests.request("POST", url, headers=post_headers, files=files)


print(response.text)

{"response":{
  "status": {
  },
  "quotaConsumed": {
    "numChars": "2789",
    "numMetadataChars": "2009"
  }
}}


### Querying 

Querying the document 

In [62]:
import requests
import json

url = "https://api.vectara.io/v1/query"

payload = json.dumps({
  "query": [
    {
      "query": "What components are used to manufecture the IOT based single phase energy meter",
      "start": 0,
      "numResults": 2,
      "contextConfig": {
        "sentences_before": 3,
        "sentences_after": 3,
        "start_tag": "<b>",
        "end_tag": "</b>"
      },
      "corpusKey": [
        {
          "corpus_id": 3
        }
      ],
      "summary": [
        {
          "max_summarized_results": 2,
          "response_lang": "en"
        }
      ]
    }
  ]
})
headers = {

  'Content-Type': 'application/json',
  'customer-id': os.getenv('customer_id'),
  'Accept': 'application/json',
  'x-api-key': os.getenv('v_api_key')
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)

{"responseSet":[{"response":[{"text":"Untitled document <b>IOT based single phase energy meter</b> Submitted by: Inam ul Rehman Submitted to:\nEngr. Muhammad Taseen\nEngr.","score":0.78914964,"metadata":[{"name":"page","value":"1"},{"name":"lang","value":"eng"},{"name":"offset","value":"0"},{"name":"len","value":"35"}],"documentIndex":0,"corpusKey":{"customerId":0,"corpusId":3,"semantics":"DEFAULT","dim":[],"metadataFilter":"","lexicalInterpolationConfig":null},"resultOffset":21,"resultLength":35},{"text":"In this project I have used te ACS712 as current sensor,\nZMPT101B as voltage sensor, load(LED Bulb or any) and esp32 . This project is not complete as my primary focus was to meaure\ncurrent ,voltage and power using sensors and send data to\ndatabase through API by creating a webserver that runs on\nesp32. Introduction <b>Single Phase Energy Meter can monitor the energy usage in your\nentire home in real time.</b> This can easily done by an ESP32\nmicrocontroller to send data(voltag

In [2]:
"""Simple example of using the Vectara REST API for searching a corpus."""

import json
import logging
import requests


def _get_query_json(customer_id: int, corpus_id: int, query_value: str):
    """Returns a query JSON."""
    query = {
        "query": [
            {
                "query": query_value,
                "num_results": 10,
                "corpus_key": [{"customer_id": customer_id, "corpus_id": corpus_id}],
            },
        ],
    }
    return json.dumps(query)


def query(customer_id: int, corpus_id: int, query_address: str, v_api_key: str, query: str):
    """Queries the data.

    Args:
        customer_id: Unique customer ID in vectara platform.
        corpus_id: ID of the corpus to which data needs to be indexed.
        query_address: Address of the querying server. e.g., api.vectara.io
        jwt_token: A valid Auth token. or v_api_key

    Returns:
        (response, True) in case of success and returns (error, False) in case of failure.

    """
    post_headers = {
        "customer-id": f"{customer_id}",
        # "Authorization": f"Bearer {jwt_token}"
        "x-api-key": v_api_key
    }

    response = requests.post(
        f"https://{query_address}/v1/query",
        data=_get_query_json(customer_id, corpus_id, query),
        verify=True,
        headers=post_headers)

    if response.status_code != 200:
        logging.error("Query failed with code %d, reason %s, text %s",
                       response.status_code,
                       response.reason,
                       response.text)
        return response, False

    message = response.json()
    if (message["status"] and
        any(status["code"] != "OK" for status in message["status"])):
        logging.error("Query failed with status: %s", message["status"])
        return message["status"], False

    for response_set in message["responseSet"]:
        for status in response_set["status"]:
            if status["code"] != "OK":
                return status, False

    return message, True

In [3]:
query(1666059633, 3, "api.vectara.io", os.getenv('v_api_key'), "What are components used to manufecture iot based single phase energy meter")

({'responseSet': [{'response': [{'text': 'IOT based single phase energy meter',
      'score': 0.7882031,
      'metadata': [{'name': 'page', 'value': '1'},
       {'name': 'lang', 'value': 'eng'},
       {'name': 'offset', 'value': '0'},
       {'name': 'len', 'value': '35'}],
      'documentIndex': 0,
      'corpusKey': {'customerId': 0,
       'corpusId': 3,
       'semantics': 'DEFAULT',
       'dim': [],
       'metadataFilter': '',
       'lexicalInterpolationConfig': None},
      'resultOffset': 0,
      'resultLength': 35},
     {'text': 'Single Phase Energy Meter can monitor the energy usage in your\nentire home in real time.',
      'score': 0.72107,
      'metadata': [{'name': 'page', 'value': '2'},
       {'name': 'lang', 'value': 'eng'},
       {'name': 'section', 'value': '10'},
       {'name': 'offset', 'value': '0'},
       {'name': 'len', 'value': '88'}],
      'documentIndex': 0,
      'corpusKey': {'customerId': 0,
       'corpusId': 3,
       'semantics': 'DEFAULT',

In [6]:
from openai import OpenAI

In [7]:
import os

In [8]:
from openai import OpenAI

In [ ]:
# openai.api_key=os.getenv('OPENAI_APi_KEY')
client = OpenAI()
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user",
        "content": 
        "Explain colud computing"
        }
    ]
)

print(completion.choices[0].message.content)